In [1]:
%pip install torch torchvision pillow


  Using cached torch-2.2.2-cp312-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached torchvision-0.17.2-cp312-cp312-macosx_10_13_x86_64.whl.metadata (6.6 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.2.2-cp312-none-macosx_10_9_x86_64.whl (150.8 MB)
Using cached torchvision-0.17.2-cp312-cp312-macosx_10_13_x86_64.whl (1.7 MB)
Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [torchvision] [torchvision]
Note: you may need to restart th

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image

# ✅ Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1), nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1), nn.BatchNorm2d(256), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, 2, 1), nn.Tanh()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

# ✅ Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Conv2d(128, 1, 4, 1, 0), nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# ✅ Transform (256x256)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ✅ Dataset Loaders from folders
import shutil
import os

# # Create class subdirectories if they don't exist
# os.makedirs("dataset/train/images", exist_ok=True)
# os.makedirs("dataset/test/images", exist_ok=True)

# # Move images to class subdirectories if needed
# for img in train_images:
#     src_path = os.path.join(original_folder, img)
#     dst_path = os.path.join("dataset/train/images", img)
#     if os.path.exists(src_path) and not os.path.exists(dst_path):
#         shutil.copy(src_path, dst_path)

# for img in test_images:
#     src_path = os.path.join(original_folder, img)
#     dst_path = os.path.join("dataset/test/images", img)
#     if os.path.exists(src_path) and not os.path.exists(dst_path):
#         shutil.copy(src_path, dst_path)

# Now use ImageFolder with the properly structured directories
train_dataset = datasets.ImageFolder("dataset/train", transform=transform)
test_dataset = datasets.ImageFolder("dataset/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# ✅ Initialize models and setup
G = Generator().to(device)
D = Discriminator().to(device)
criterion = nn.MSELoss()
optimizer_G = optim.Adam(G.parameters(), lr=0.0002)
optimizer_D = optim.Adam(D.parameters(), lr=0.0002)

# ✅ Training Loop
num_epochs = 100
os.makedirs("outputs", exist_ok=True)

for epoch in range(num_epochs):
    G.train()
    for i, (imgs, _) in enumerate(train_loader):
        real_imgs = imgs.to(device)
        noisy_imgs = real_imgs + 0.05 * torch.randn_like(real_imgs)

        # Train Generator
        optimizer_G.zero_grad()
        gen_imgs = G(noisy_imgs)
        validity = D(gen_imgs)
        g_loss = criterion(validity, torch.ones_like(validity))
        g_loss.backward()
        optimizer_G.step()

        # Train Discriminator
        optimizer_D.zero_grad()
        real_validity = D(real_imgs)
        fake_validity = D(gen_imgs.detach())
        d_loss = criterion(real_validity, torch.ones_like(real_validity)) + \
                 criterion(fake_validity, torch.zeros_like(fake_validity))
        d_loss.backward()
        optimizer_D.step()

        if i % 100 == 0:
            print(f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(train_loader)}] "
                  f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")

    # ✅ Save generated images using test set
    G.eval()
    with torch.no_grad():
        for test_imgs, _ in test_loader:
            test_imgs = test_imgs.to(device)
            noisy_test = test_imgs + 0.05 * torch.randn_like(test_imgs)
            denoised = G(noisy_test)
            save_image(denoised[:16], f"outputs/epoch_{epoch}.png", nrow=4, normalize=True)
            break  # Save only first batch


[Epoch 0/100] [Batch 0/235] [D loss: 0.5293] [G loss: 0.2004]


KeyboardInterrupt: 